In [426]:
#Lloyd : All data pre cached here
#Lloyd : Norgate can be swapped out here

import pandas as pd 

STOCKS         : pd = pd.read_pickle("stocks_alpaca.pkl")
STOCKS_1         : pd = pd.read_pickle("stocks_nasdaq.pkl")
PACKED_OPTIONS : pd = pd.read_pickle("options_packed.pkl")
EARNINGS       : pd = pd.read_pickle("earnings.pkl")

print(STOCKS[ STOCKS.Symbol == 'ADM']['Close'])
print(STOCKS_1[ STOCKS_1.Symbol == 'ADM']['Close'])

# print(STOCKS[ STOCKS.Symbol == 'AMP'])
# print(PACKED_OPTIONS)
# print(EARNINGS)

Time
2021-11-05T04:00:00Z    64.49
2021-11-08T05:00:00Z    63.80
2021-11-09T05:00:00Z    64.43
2021-11-10T05:00:00Z    65.39
2021-11-11T05:00:00Z    66.14
                        ...  
2022-10-31T04:00:00Z    96.98
2022-11-01T04:00:00Z    97.53
2022-11-02T04:00:00Z    95.80
2022-11-03T04:00:00Z    96.28
2022-11-04T04:00:00Z    95.19
Name: Close, Length: 252, dtype: float64
Date
01/21/2022    68.94
01/24/2022    68.59
01/25/2022    68.37
01/26/2022    72.27
01/27/2022    74.20
              ...  
10/31/2022    96.98
11/01/2022    97.53
11/02/2022    95.80
11/03/2022    96.28
11/04/2022    95.19
Name: Close, Length: 200, dtype: float64


In [427]:
#092321 Modify the TDAmeritrade API get refresh token algo
#061121 Hide the TDAmeritrade API credentials
#022021 Add controal group to compare the significance of WOO rating  
#012721 Adjust earnings date and expiration date comparison method. 
#011221 Add code to request option contracts data in one shot
#010721 Add time delay due to increased number of stocks and TOS limitation.
#123120 Add try and except to report function for updating earnings date
#121720 Replace RSI from Alpha-Vantage with local function (Alpha-Vantage does not work properly) 
#110320 Fixed bugs in RSI update frequency and XLC list building.
#102220 Add a step to check if the algo uses the most recent data: data_test function
#100220 Add a row of 'No Recommendations' to show if the output is empty
#090820 Replace source of S&P 500 members' symbols
#072720 Add Google Spreadsheet update for AppSheet

# using Norgate data
# using Yahoo Finance Earning Calendar


import norgatedata
priceadjust = norgatedata.StockPriceAdjustmentType.TOTALRETURN 
padding_setting = norgatedata.PaddingType.NONE   
timeseriesformat = 'pandas-dataframe'


#from alpha_vantage.timeseries import TimeSeries
#import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, date
import requests
import pandas as pd
pd.options.mode.chained_assignment = None

import time
#import quandl
import sys
import math
from tqdm import tqdm, trange
#from alpha_vantage.techindicators import TechIndicators
#from datapackage import Package
#import urllib.request, json
#from pymemcache.client import base

## Changed

In [428]:
support1 = 0
support2 = 0
resistance1 = 0
resistance2 = 0

In [429]:
# def all_time_rsi_values(stock_para):
#     ts = TechIndicators(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
#     data, meta_data = ts.get_rsi(symbol=stock_para)
#     return data['RSI'][-1]

In [430]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# def get_earnings_date():
#     """
#     Get earnings dates for a week from today by scraping data from https://api.earningscalendar.net/
#     """
#     start_date = date.today()
#     end_date = date.today()+timedelta(5)

#     list_4_today=list()
#     for single_date in daterange(start_date, end_date):
#         time.sleep(5)
#         x=single_date.strftime("%Y%m%d")
#         urlto_search="https://api.earningscalendar.net/?date="+x
#         with urllib.request.urlopen(urlto_search) as url:
#             data = json.loads(url.read().decode())
#             for dat in data:
#                 list_4_today.append(dat["ticker"])
#     return list_4_today


In [431]:

# def get_all_data(symbol_para):
#     symbol = symbol_para
#     start_date = '2015-01-01' #ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
#     try:
#         #print(symbol_para)
#         pricedata_dataframe = norgatedata.price_timeseries(
#                     symbol,
#                     stock_price_adjustment_setting = priceadjust,
#                     padding_setting = padding_setting,
#                     start_date = start_date,
#                     format=timeseriesformat)
#         company_name = norgatedata.security_name(symbol)[:-7]
#     except:
#         return pd.DataFrame(), str()
#     myData = pricedata_dataframe.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
#     myData['20d'] = np.round(myData.Close.rolling(window =20, center = False).mean(),2)
#     myData['50d'] = np.round(myData.Close.rolling(window =50, center = False).mean(),2)
#     myData['200d'] = np.round(myData.Close.rolling(window =200, center = False).mean(),2)
#     #print(myData.tail(5))
    
#     return (myData, company_name)


In [432]:
#Lloyd : stocks built in first cell
#Lloyd : company names can be appended after

def get_all_data(symbol):
    myData = STOCKS[ STOCKS.Symbol == symbol ]
    myData['20d'] = np.round(myData.Close.rolling(window =20, center = False).mean(),2)
    myData['50d'] = np.round(myData.Close.rolling(window =50, center = False).mean(),2)
    myData['200d'] = np.round(myData.Close.rolling(window =200, center = False).mean(),2)
    
    return (myData, 'N/A')

In [433]:
#SMA rating
def SMA_rating(data_simple):
#     start_date = date(1998,1,2)
#     yesterday = (datetime.now() - timedelta(days=1))
#     end_date = date(yesterday.year,yesterday.month,yesterday.day)
    sma_rating = 0
    index = len(data_simple.index)-1
    if data_simple['Close'][index] > data_simple['50d'][index] > data_simple['200d'][index]:
        sma_rating=1
    elif data_simple['50d'][index] > data_simple['Close'][index]> data_simple['200d'][index]:
        sma_rating=2
    elif data_simple['50d'][index] > data_simple['200d'][index] > data_simple['Close'][index]:
        sma_rating=3
    elif data_simple['200d'][index] > data_simple['50d'][index] > data_simple['Close'][index]:
        sma_rating=4
    elif data_simple['200d'][index]> data_simple['Close'][index] > data_simple['50d'][index]:
        sma_rating=5
    elif data_simple['Close'][index] > data_simple['200d'][index] > data_simple['50d'][index]:
        sma_rating=6
    elif data_simple['Close'][index] > data_simple['50d'][index] > data_simple['200d'][index]:
        sma_rating=7
    return sma_rating

In [434]:
def MACD_rating(data_simple):
        data_simple['26 ema']=data_simple.Close.ewm(span=26).mean()
        data_simple['12 ema']=data_simple.Close.ewm(span=12).mean()
        data_simple['MACD'] = (data_simple['12 ema'] - data_simple['26 ema'])
        data_simple['signal_line']=data_simple.MACD.ewm(span=9).mean()
        data_simple['hist']=(data_simple['MACD']-data_simple['signal_line'])
        
        #MACD rating
        index = len(data_simple)-1
        slope = 0
        macd_rating = 0
        
        #print("Current MACD:",data_simple['MACD'][index])
        #print("Previous day MACD",data_simple['MACD'][index-1])
        
        #Condition for the slope
        if((data_simple['MACD'][index] - data_simple['MACD'][index-1])>0):
            slope = 1
        elif((data_simple['MACD'][index] - data_simple['MACD'][index-1])<0):
            slope = -1
            
        #Condition for the MACD rating
        if data_simple['MACD'][index] > data_simple['signal_line'][index] and data_simple['signal_line'][index] > 0:
            macd_rating=1
        elif data_simple['signal_line'][index] > data_simple['MACD'][index] and data_simple['MACD'][index] > 0:
            macd_rating=2
        elif data_simple['MACD'][index] < 0 and 0 < data_simple['signal_line'][index]:
            macd_rating=3
        elif data_simple['MACD'][index] < data_simple['signal_line'][index] and data_simple['signal_line'][index]< 0:
            macd_rating=4
        elif data_simple['signal_line'][index] < data_simple['MACD'][index] and data_simple['MACD'][index] < 0:
            macd_rating=5
        elif data_simple['MACD'][index] > 0 and 0 > data_simple['signal_line'][index]:
            macd_rating=6
        return macd_rating,slope

In [435]:
def daily_rsi_values(stock, data_simple, n=14):
#     ts = TechIndicators(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
#     try:
#         data, meta_data = ts.get_rsi(symbol=stock_para, interval='daily') # interval = '1min'
#         print('{}\'s RSI is {}.'.format(stock_para,data['RSI'][-2]))
#     except:
#         return -1
#     if(data.empty):
#         return -1
#     if(data['RSI'][-1]>70 or data['RSI'][-1]<30):
#         ret_val = -1
#     else:
#         ret_val = 1
#     return ret_val
   
    #def RSI(self, prices, n=14):
    
    rsi = 0
    index = len(data_simple.index)-1
    prices = data_simple['Close']
    symbol = stock
    #print(symbol)
    
    deltas = np.diff(prices)
    #if index >360:
    #    seed = deltas[index-360 : index-360 + n+1]
    #else:
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n+1, len(prices)):
        delta = deltas[i-1]  # The diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)
        
    if(rsi[-1] == False):
        print('RSI computing error found.')
        return -1
    elif(rsi[-1]>70 or rsi[-1]<30):
        #print('{}\'s RSI is {}.'.format(symbol,rsi[-1]))
        rsi_rating = -1
    else:
        rsi_rating = 1
        #print('{}\'s RSI is {}.'.format(symbol,rsi[-1]))

    return rsi_rating


# 3/5/2020 Charlie suggests this label was wrong. He recommends RSI>40 for 1, and others for -1.

## Changed

In [436]:
## Global Declarations

reversal = 3 #reversal amount set to 3 as default

# (price range, box size)
#function will provide box sizes according to traditional method
box_ranges = [(.25,.0625),
              (1,.125), 
              (5,.25),
              (20,.5),
              (100,1),
              (200,2),
              (500,4),
              (1000,5),
              (25000,50),
              (sys.maxsize,500)]

#to check and update box size according to current value of stock
def updateBoxSize(price):
    for i in range (len(box_ranges)):
        if price < box_ranges[i][0]: #Zhen: Does it return multiple values?
            return box_ranges[i][1] 
    return None


#using alpha_vantage because quandl gives only data till March 2018

#if current trend is x update x column appropriately or move to o column if needed
def updateX(item, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes):
    box_size = updateBoxSize(list1[-1])
   # print(math.floor(item["High"]))
    if ( math.floor(item["High"]) >= list1[-1]+box_size ):
        list1[-1] = math.floor(item["High"])
    #   print("Updated the x value to:"+str(list1[-1]))
        numberofXBoxes += 1
    elif ( math.ceil(item["Low"]) <= list1[-1]-reversal*box_size):
        list2.append(math.ceil(item["Low"]))
    #   print("Updated the o value to:"+str(list2[-1]))
        current_trend = 'o'
        numberofOBoxes += 1
    return current_trend

#if current trend is o update o column appropriately or move to x column if needed
def updateO(item, list1, list2, box_size, reversal, current_trend,  numberofXBoxes, numberofOBoxes):
    box_size = updateBoxSize(list2[-1])
    if ( math.ceil(item["Low"]) <= list2[-1]-box_size ):
        list2[-1] = math.ceil(item["Low"])
#        print("Updated the o value to:"+str(list2[-1]))
        numberofOBoxes += 1
    elif ( math.floor(item["High"]) >= list2[-1]+reversal*box_size):
        list1.append(math.floor(item["High"]))
 #       print("Updated the x value to:"+str(list1[-1]))
        current_trend = 'x'
        numberofXBoxes += 1
    return current_trend

#create the point and figure from scratch

def pointAndFigureCreate(box_size,current_trend, list1, list2, stockhilowdata, numberofXBoxes, numberofOBoxes):
    #iterate over each date for the stock and create x or o columns
    for index,row in stockhilowdata.iloc[0:].iterrows():
        if current_trend == 'x':
        #   print("The price is:"+str(row["High"]))
            current_trend = updateX(row, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes)
        elif current_trend=='o':
            current_trend = updateO(row, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes)
          #  print("The price is:"+str(row["Low"]))
    return current_trend

#check for a continous triple top pattern
def continous_triple_top(current_trend, list1):
    if(len(list1)<3):
        return -1
    ret_val = 0
    if current_trend == 'x':
        if((list1[-1]> list1[-2]) and (list1[-2] == list1[-3])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val
        
#check for a continous triple bottom pattern
def continous_triple_bottom(current_trend, list2):
    if(len(list2)<3):
        return -1
    ret_val = 0
    if current_trend == 'o':
        if((list2[-1] < list2[-2]) and (list2[-2] == list2[-3])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

#check for a continous double top pattern
def continous_double_top(current_trend, list1):
    if(len(list1)<2):
        return -1
    ret_val = 0
    if current_trend=='x':
        if(list1[-1]> list1[-2]):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

#check for a continous double bottom pattern
def continous_double_bottom(current_trend, list2):
    if(len(list2)<2):
        return -1
    ret_val = 0
    if current_trend == 'o':
        if(list2[-1]< list2[-2]):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def continous_quadruple_top(current_trend, list1):
    if(len(list1)<4):
        return -1
    ret_val = 0
    if current_trend == 'x' and len(list1)>3:
        if((list1[-1] > list1[-2]) and (list1[-2] == list1[-3] == list1[-4])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def continous_quadruple_bottom(current_trend, list2):
    if(len(list2)<4):
        return -1
    ret_val = 0
    if current_trend == 'o' and len(list2)>3:
        if((list2[-1] < list2[-2]) and (list2[-2] == list2[-3] == list2[-4])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

import itertools
def spread_triple_top(current_trend, list1):
    ret_val = -1
    if current_trend == 'x' and len(list1)>7:
        iterprev = 0
        resistance = 0
        notstt = True
        for iter in list1[-2:-7:1]:
            if iterprev == iter:
                resistance = iter
                notstt = False
            if resistance!=0 and iter > resistance:
                notstt = True
                break
            iterprev = iter
        if(notstt == False):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def spread_triple_bottom(current_trend, list2):
    ret_val = -1
    if current_trend == 'o' and len(list2)>7:
        iterprev = 0
        support = 0
        notstb = True
        for iter in list2[-2:-7:1]:
            if iterprev == iter:
                support = iter
                notstb = False
            if support!=0 and iter<resistance:
                notstb = True
                break
            iterprev = iter
        if(notstb == False):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def get_support_levels(list2):
    if(len(list2)>=2):
        return list2[-1],list2[-2]
    elif (len(list2)==1):
        return list2[-1],0
    else:
        return 0,0

def get_resistance_levels(list1):
    if(len(list1)>=2):
        return list1[-1],list1[-2]
    elif (len(list1)==1):
        return list1[-1],0
    else:
        return 0,0
    
    
def PnFMain(myData):
    #list1 stores the highest value of each x column by index
    #list2 stores the lowest value of each o column by index
    list1 = []
    list2 = []

    #current_trend stores the value of the most recent trend. we start off with x as default
    current_trend = 'x'

    numberofXBoxes = 1 # to get number of x boxes in all
    numberofOBoxes = 0 # to get number of o boxes in all

    #set box size according to price on day 1
    box_size = updateBoxSize(myData["High"].iloc[0])
    #append price on day 1 to list1 as we stfart with x
    list1.append(math.floor(myData["High"].iloc[0]))
    current_trend = pointAndFigureCreate(box_size, current_trend, list1, list2, myData, numberofXBoxes, numberofOBoxes)
    ret_val_triple_top = continous_triple_top(current_trend,list1)
    ret_val_triple_bottom = continous_triple_bottom(current_trend,list2)
    ret_val_double_top = continous_double_top(current_trend,list1)
    ret_val_double_bottom = continous_double_bottom(current_trend,list2)
    ret_val_quadruple_top = continous_quadruple_top(current_trend,list1)
    ret_val_quadruple_bottom = continous_quadruple_bottom(current_trend,list2)
    ret_val_spread_triple_top = spread_triple_top(current_trend,list1)
    ret_val_spread_triple_bottom = spread_triple_bottom(current_trend,list2)
    
    global support1
    global support2
    global resistance1
    global resistance2
    
    support1,support2 = get_support_levels(list2)
    resistance1,resistance2 = get_resistance_levels(list1)
    
    if ret_val_quadruple_top == 1:
        return 'a'
    elif ret_val_triple_top == 1:
        return 'b'
    elif ret_val_double_top == 1:
        return 'c'
    elif ret_val_quadruple_bottom == 1:
        return 'd'
    elif ret_val_triple_bottom == 1:
        return 'e'
    elif ret_val_double_bottom == 1:
        return 'f'
    else:
        return None
    

# Initialize the instruments

In [437]:
financial_list=list()
energy_list=list()
industrial_list=list()
comm_services_list=list()
materials_list=list()
health_list=list()
consumer_list=list()
technology_list=list()
consumer_stap_list=list()

sector_etf_list=['XLF','XLE','XLI','XLC','XLB','XLV','XLY','XLK','XLP']
sector_sma_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_macd_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_macd_slope={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_pnf_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_rsi_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_etf_to_names={'XLF':financial_list,'XLE':energy_list,'XLI':industrial_list,'XLC':comm_services_list,'XLB':materials_list,'XLV':health_list,'XLY':consumer_list,'XLK':technology_list,'XLP':consumer_stap_list}


"""sector_etf_list=['XLF']
sector_sma_rating={'XLF':0}
sector_macd_rating={'XLF':0}
sector_macd_slope={'XLF':0}
sector_pnf_rating={'XLF':0}
sector_rsi_rating={'XLF':0}
sector_etf_to_names={'XLF':financial_list}"""


"sector_etf_list=['XLF']\nsector_sma_rating={'XLF':0}\nsector_macd_rating={'XLF':0}\nsector_macd_slope={'XLF':0}\nsector_pnf_rating={'XLF':0}\nsector_rsi_rating={'XLF':0}\nsector_etf_to_names={'XLF':financial_list}"

In [438]:
# This code uses datahub.io to collect S&P500 members' symbols.
#
#package = Package('https://datahub.io/core/s-and-p-500-companies/datapackage.json')
# for resource in package.resources:
#     if resource.descriptor['datahub']['type'] == 'derived/csv':
#         my_list=resource.read()
# df = pd.DataFrame(my_list, columns=['Symbol','Name','Sector'])



# This code uses wikipedia.org to collect S&P members' symbols

import pandas as pd
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', header = 0)
df1 = table[0]

df = df1[['Symbol', 'Security', 'GICS Sector']]

df = df.rename(columns = {"GICS Sector":'Sector'})

In [439]:
for i in range(0,len(df)):
    if df['Sector'][i]=='Industrials':
        industrial_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Health Care':
        health_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Energy':
        energy_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Financials':
        financial_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Information Technology':
        technology_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Consumer Discretionary':
        consumer_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Materials':
        materials_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Communication Services':
        comm_services_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Consumer Staples':
        consumer_stap_list.append(df['Symbol'][i])

In [440]:
# def data_test():
#     """Check the integrety of data."""
#     from pandas.tseries.offsets import BDay

#     today = datetime.today()
#     LastBday = today - BDay(1)

#     print('The last business date is {}.'.format(LastBday.date()))
#     testData, symbol = get_all_data('SPY') 
#     #print(type(testData.index[-1]))
#     data_date = datetime.strptime(testData.index[-1], "%Y-%m-%d %H:%M:%S")
#     #print(data_date.date())
#     print('The last date in data is {}.'.format(data_date.date())) #format(data_date) #.date())

#     if data_date.date() != LastBday.date():
#         print('\n ###################################################################### \n \
# #  The last business day\'s data is incorrect. Check the data source. # \n\
#  ######################################################################')
#         raise SystemExit
#     else:
#         print('The last business day\'s data is correct. You can proceed.')


In [441]:
# data_test()

In [442]:
'''Get data for applying ratings.'''
df_rated_now = pd.DataFrame()

for sector in tqdm(sector_etf_list):
    print(sector+':', end = '')
    for i in range(0,len(sector_etf_to_names[sector])):        
        myData, company_name = get_all_data(sector_etf_to_names[sector][i])            
        if(not myData.empty):

            if (not myData.Close.isnull().values.any()):

                myData = myData.tail(200)

                macd_rating,slope = MACD_rating(myData)
                df_rated_now = df_rated_now.append({'Stock Symbol':sector_etf_to_names[sector][i], 
                                                'Company Name': company_name, 
                                                'SMA':SMA_rating(myData), 
                                                'MACD':macd_rating,
                                                'MACD Slope':slope, 
                                                'PnF':PnFMain(myData), 
                                                'RSI':daily_rsi_values(sector_etf_to_names[sector][i], myData), 
                                                'Support1':support1, 'Support2':support2,'Resistance1':resistance1, 'Resistance2':resistance2 }, 
                                                ignore_index=True)
        else:
            print('Data for stock is not present in the API')
    #print(df_rated_now)

print('Done')


# df_rated_now[df_rated_now['Stock Symbol']=='FBHS']

  0%|          | 0/9 [00:00<?, ?it/s]

XLF:

 11%|█         | 1/9 [00:02<00:16,  2.05s/it]

XLE:

 22%|██▏       | 2/9 [00:02<00:11,  1.66s/it]

...

XLK:

 89%|████████▉ | 8/9 [00:12<00:01,  1.77s/it]

XLP:

100%|██████████| 9/9 [00:13<00:00,  1.50s/it]

Done


In [443]:
# df_rated_now.to_csv('Rated_Stocks_All.csv', index=False)


In [444]:
# Define contral groupo
# Randomly choose 20 stocks from the full list of stocks

df_control = df_rated_now.sample(n=40)

df_control = df_control.rename(columns={"Stock Symbol":"Symbol"})

# df_control.to_csv(r'C:\Users\User\Box\WOOatUB\YashaAbhinavRohit\Stock_List_Control.csv')

# df_control.to_csv(r'C:\Users\User\Box\WOOatUB\YashaAbhinavRohit\ContractsData\\'+ datetime.today().strftime('%Y-%m-%d')+'_control''.csv')

In [445]:
#from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import time
#import quandl
import sys
import math
#from alpha_vantage.techindicators import TechIndicators
#from datapackage import Package
import urllib.request, json
#from pymemcache.client import base

# Location = r'Rated_Stocks_All.csv'
# df_rated_now = pd.read_csv(Location)
df_rated_now.head()

,Company Name,MACD,MACD Slope,PnF,RSI,Resistance1,Resistance2,SMA,Stock Symbol,Support1,Support2
0,N/A,1.0,1.0,c,-1.0,68.0,60.0,1.0,AFL,57.0,57.0
1,N/A,4.0,1.0,c,1.0,129.0,127.0,6.0,ALL,120.0,117.0
2,N/A,5.0,1.0,None,1.0,146.0,149.0,4.0,AXP,139.0,133.0
3,N/A,1.0,1.0,c,-1.0,58.0,51.0,6.0,AIG,48.0,48.0
4,N/A,1.0,1.0,None,1.0,319.0,319.0,1.0,AMP,302.0,258.0


In [446]:
def get_final_list():
    final_list = []
    pnf_list = ['a','b','c']
    for index,row in df_rated_now.iloc[0:].iterrows():
        if row['MACD'] == 1 and row['MACD Slope'] == 1 and row['SMA'] == 1 and row['PnF'] in pnf_list and row['RSI'] == 1:
            row_data = {'Symbol':row['Stock Symbol'] , 'Company Name':row['Company Name'], 'Score':row['PnF'], 'Support1':row['Support1'],'Support2':row['Support2'],'Resistance1':row['Resistance1'],'Resistance2':row['Resistance2']}
            final_list.append(row_data)
    final_df = pd.DataFrame(final_list)
    if not final_df.empty:
        final_df['SymbolKey'] = final_df['Symbol']
        final_df.set_index('SymbolKey', inplace = True)
    else:
        return pd.DataFrame()
    return final_df

In [447]:
def get_close_price(stock_list):
    '''For Point and Figure rating'''
    close_price = list()
    for row in stock_list.iterrows():
        symbol = row[1]['Symbol']
        #print("Symbol is:", symbol)
            #symbol = symbol_para
        start_date = '2010-01-01' #ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
        try:
            pricedata_dataframe = norgatedata.price_timeseries(
                    symbol,
                    stock_price_adjustment_setting = priceadjust,
                    padding_setting = padding_setting,
                    start_date = start_date,
                    format=timeseriesformat)
            close_price.append(pricedata_dataframe['Close'].iloc[-1])
        except:
            close_price.append(-1)
    stock_list['Close Price']=close_price
    return stock_list

In [448]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [449]:
# def next_earnings_date(symbol):
#     # Use Yahoo Finance Earnings Calendar https://github.com/wenboyu2/yahoo-earnings-calendar
#     from datetime import datetime
#     from yahoo_earnings_calendar import YahooEarningsCalendar
#     my_custom_delay_s = 0.5
#     #try:
#     yec = YahooEarningsCalendar(my_custom_delay_s)# Returns the next earnings date of BOX in Unix timestamp
#     ts = yec.get_next_earnings_date(symbol)
#     return datetime.fromtimestamp(ts)
#     #except:
#         #print(symbol)
#     #    print('Situation: ' + symbol + 'Invalid Symbol or Unavailable Earnings Date')
#         #return datetime.fromtimestamp(ts)
        

In [450]:
# Lloyd : will fetch from dataframe in first cell
# 
def next_earnings_date(symbol):
    # Use Yahoo Finance Earnings Calendar https://github.com/wenboyu2/yahoo-earnings-calendar
    return  EARNINGS[ EARNINGS.index == symbol].iloc[0]['date'] 
        

In [451]:
next_earnings_date('GOOGL')

Timestamp('2023-01-30 16:00:00')

In [452]:
df_top_five_global = pd.DataFrame()

In [453]:
# import requests
# import json
# import csv
# import pandas as pd
# from pandas.io.json import json_normalize
# from datetime import datetime  
# from datetime import timedelta
# import logging

# import config as cf


# # Following code makes relevant tokens necessary to conduct TD ameritrade activities on behalf of Zhen Liu's personal account
# # Note refresh token is valid for 90 days. It is used to generate auth token which is only valid for 30 mins
# def get_auth_token():
#     auth_params = {'grant_type':'refresh_token', 'refresh_token':cf.refresh_token, 'client_id': cf.client_id}
#     auth_api_url = 'https://api.tdameritrade.com/v1/oauth2/token'
#     headers = {'Content-Type': "application/x-www-form-urlencoded"}
#     data =  requests.post(auth_api_url, headers = headers, data=auth_params).json()
#     #print(data)
#     result = "Bearer "+ data['access_token']

#     return result

# # Following code will get the option chain data using the authorized token
# # Parameters: call_or_put: 'CALL'/'PUT'; symbol: string with stock symbol from passed in stock list
# def get_raw_option_data(call_or_put, symbol):
#     url = 'https://api.tdameritrade.com/v1/marketdata/chains'
#     strike_count = '100' #need to figure out how to get ALL or just use an arbitraliy huge num
#     strategy = 'SINGLE'
#     authorization_header = {'Authorization': get_auth_token()}
#     pay = {'symbol':symbol,'strikeCount':strike_count , 'strategy':strategy}
#     if call_or_put == "CALL":
#         return [requests.get(url, params = pay, headers = authorization_header).json()['callExpDateMap'], 
#                 requests.get(url, params = pay, headers = authorization_header).json()['underlyingPrice']]
#     elif call_or_put == "PUT":
#         return [requests.get(url, params = pay, headers = authorization_header).json()['putExpDateMap'], requests.get(url, params = pay, headers = authorization_header).json()['underlyingPrice']]



# # Following code does the filtering of the options data based on our chosen parameters
# def processing(symbol, Option_Data, supp1,supp2, params, call_or_put):
#     data = Option_Data[symbol]
#     agg_data = {}
#     date_keys = []
#     for (k,v) in data[0].items():
#         date_keys.append([k,list(v.keys())])
#         agg_data[k] = v
        
#     #If there is no data regarding some symbol in API then just return an empty dataframe
#     if not date_keys:
#         return pd.DataFrame()
        
#     target_list = []
#     stock_price = data[1]
    
#     if call_or_put == "CALL":
#         for i in range(0,len(date_keys)):
#             for j in range(0, len(date_keys[i][1])):
#                 if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
#                     continue
#                 if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
#                     continue
#                 if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
#                     continue
#                 if (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price < params['%OTM_lim']: # Percent OTM Requirement!
#                     continue
#                 if 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'] < params['prob_OTM_lim']: # Prob OTM Requirement!
#                     continue
#                 support = 0
#                 if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']<supp1 and agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']>supp2:
#                     support = 1
#                 # construct a dictionary object characteristics we want and append to list
#                 target = {'stock': symbol,
#                          'contract': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
#                          'strike_price':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
#                          'stock_price':stock_price,
#                          'days_to_exp':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],
#                          'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
#                          'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
#                          'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
#                          'prob_OTM': 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'],
#                          '% OTM': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price,
#                          'cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'])/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
#                          'max_cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']+((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
#                          'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*2,
#                          'annual_return': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'] * 365 * 100)/((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']) * (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'])),
#                          'btwn_supports':support}
#                 target_list.append(target)
#     elif call_or_put == "PUT":
#         for i in range(0,len(date_keys)):
#             for j in range(0, len(date_keys[i][1])):
#                 if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or \
#                 agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
#                     continue
#                 if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
#                     continue
#                 if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
#                     continue
#                 if (stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']) / \
#                 stock_price < params['%OTM_lim']: 
#                     # Percent OTM Requirement!
#                     continue
#                 prob_otm = 1 + float(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'])
#                 #print("The prob of OTM is", round(prob_otm, 2))
#                 #print("The otm limit is", params['prob_OTM_lim'])
#                 if prob_otm < params['prob_OTM_lim']: # Prob OTM Requirement!      + or -???
#                     #print("The prob of OTM", round(prob_otm, 2))
#                     continue
#                 if (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] - \
#                           max(0, (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - \
#                             stock_price)))/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']- \
#                                            agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']) * 365 * 100/\
#                            agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < \
#                             params['annual_ptnl_ret_lim']: # Covered return Requirement! 
#                     continue
#                 if float(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'])*(-2) > \
#                 params['prob_touch_lim']: # Probability of touching parameter      + or - ???
#                     continue
#                 support = 0
#                 if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']<supp1 and agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']>supp2:
#                     support = 1
# #Commented code checks for 24% annual return. If required in future
# #                 value = (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']*365*100)/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']*agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'])
# #                 if value<15:
# #                     continue
    
#                 # Construct a dictionary object characteristics we want and append to list
#                 target = {'stock':symbol,
#                          'contract':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
#                          'strike_price':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
#                          'stock_price':stock_price,
#                          'days_to_exp':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],
#                          'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
#                          'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
#                          'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
#                          'prob_OTM': 1 + float(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']), # + or - ???
#                          '% OTM': 1-agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']/stock_price,
#                          'ptnl_ret': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] - \
#                           max(0, (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - \
#                             stock_price)))/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']- \
#                                            agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']),
#                          'annual_ptnl_ret': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] - \
#                           max(0, (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - \
#                             stock_price)))/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']- \
#                                            agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']) * 365 * 100/\
#                            agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],

#                          'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*(-2),
#                          'btwn_supports': support}# + or - ???
                
# #                 print("The value of accepted delta",target['prob_OTM'])
#                 target_list.append(target)
#     df = pd.DataFrame(target_list)
#     if(df.empty):
#         print('Got empty')
#         return df
#     df = df.set_index('contract', drop = True)
#     return df



# # Following function returns price data for a stock which can be used to construct a P&F chart
# # def return_candle_data(symbol):
# #     candle_url = 'https://api.tdameritrade.com/v1/marketdata/'+symbol+'/pricehistory'
# #     candle_header = {'Authorization': get_auth_token()}
# #     # Vary the properties below depending on how often you want the data
# #     # Ideally, we want to set up a web socket to continuously stream this data?
# #     period_type = 'day'
# #     period = 2
# #     freq_type = 'minute'
# #     freq = 1
# #     candle_payload = {'periodType': period_type, 'period': period, 'frequencyType': freq_type, 'frequency': freq}
# #     raw_candle_data = requests.get(candle_url, params = candle_payload, headers = candle_header).json()
# #     df = pd.DataFrame(raw_candle_data['candles'])
# #     return df


# def report(stock_list, Option_Data, put_parameters):
#     # Select by WOO ratings and earnings dates criteria
    
#     df_top_five_all =  pd.DataFrame()
    
#     d0 = date.today()
    
#     for row in tqdm(stock_list.iterrows()):
#         item = row[1]['Symbol']
#         supp1 = row[1]['Support1']
#         supp2 = row[1]['Support2']
#         name = row[1]['Company Name']
#         print(item, end=" ")
#         df = processing(item, Option_Data, supp1,supp2, put_parameters, "PUT")
#         if(not df.empty):
#             df_top_five = df.sort_values(by=['annual_ptnl_ret'], ascending=False)
#             df_top_five = df_top_five.head(5)
#             df_top_five['company name'] = name
#             if df_top_five_all.empty:
#                 df_top_five_all = df_top_five
#             else:
#                 df_top_five_all = df_top_five_all.append(df_top_five)
#         #time.sleep(5)
                
#     if(not df_top_five_all.empty):
#         df_top_five_all = df_top_five_all.sort_values(by=['btwn_supports','stock','annual_ptnl_ret'], ascending=[False,True,False])
        
#         df_top_five_all['Days to Earnings'] = '!' # In case earnings date data is not avaialbe, avoid reporting the contracts.
        
#         df_top_five_all['Risk level'] = put_parameters['label']
        
#         for index, row in df_top_five_all.iterrows():
            
#             try:
#                 ed = next_earnings_date(row['stock']).date()
#                 delta = ed - d0 #compute dates from today to earnings date
#                 #print(delta.days)
#                 print('The next earnings date of {} is {}.'.format(row['stock'],ed))
           
#                 #if delta.days>0: # Yahoo may report earning dates that just passed.
#                 d_days = delta.days - row['days_to_exp']                

#                 if (d_days <= 0):
#                     df_top_five_all = df_top_five_all.drop(index, axis=0)
#                 else:
#                     df_top_five_all.at[index, 'Days to Earnings']= delta.days
#             except:
#                 print('Error happens in finding the next earnings data.')
#                 df_top_five_all = df_top_five_all.drop(index, axis=0)
#                 pass

#     else:
#         print("No contracts available in the {} criteria.".format(put_parameters['label']))
    
#     return df_top_five_all    
               
        
    
        

In [454]:
import requests
import json
import csv
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime  
from datetime import timedelta
import logging

import config as cf


# Following code will get the option chain data using the authorized token
# Parameters: call_or_put: 'CALL'/'PUT'; symbol: string with stock symbol from passed in stock list
def get_raw_option_data(call_or_put, symbol):

    option : dict = json.loads( PACKED_OPTIONS[ PACKED_OPTIONS["Symbol"] == symbol ]['Json'].iloc[0] )

    return option


# Following code does the filtering of the options data based on our chosen parameters
def processing(symbol, Option_Data, supp1,supp2, params, call_or_put):
    data = Option_Data[symbol]
    agg_data = {}
    date_keys = []
    for (k,v) in data[0].items():
        date_keys.append([k,list(v.keys())])
        agg_data[k] = v
        
    #If there is no data regarding some symbol in API then just return an empty dataframe
    if not date_keys:
        return pd.DataFrame()
        
    target_list = []
    stock_price = data[1]
    
    if call_or_put == "CALL":
        for i in range(0,len(date_keys)):
            for j in range(0, len(date_keys[i][1])):
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
                    continue
                if (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price < params['%OTM_lim']: # Percent OTM Requirement!
                    continue
                if 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'] < params['prob_OTM_lim']: # Prob OTM Requirement!
                    continue
                support = 0
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']<supp1 and agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']>supp2:
                    support = 1
                # construct a dictionary object characteristics we want and append to list
                target = {'stock': symbol,
                         'contract': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
                         'strike_price':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
                         'stock_price':stock_price,
                         'days_to_exp':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],
                         'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
                         'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
                         'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
                         'prob_OTM': 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'],
                         '% OTM': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price,
                         'cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'])/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'max_cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']+((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*2,
                         'annual_return': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'] * 365 * 100)/((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']) * (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'])),
                         'btwn_supports':support}
                target_list.append(target)
    elif call_or_put == "PUT":
        for i in range(0,len(date_keys)):
            for j in range(0, len(date_keys[i][1])):
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or \
                agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
                    continue
                if (stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']) / \
                stock_price < params['%OTM_lim']: 
                    # Percent OTM Requirement!
                    continue
                prob_otm = 1 + float(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'])
                #print("The prob of OTM is", round(prob_otm, 2))
                #print("The otm limit is", params['prob_OTM_lim'])
                if prob_otm < params['prob_OTM_lim']: # Prob OTM Requirement!      + or -???
                    #print("The prob of OTM", round(prob_otm, 2))
                    continue
                if (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] - \
                          max(0, (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - \
                            stock_price)))/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']- \
                                           agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']) * 365 * 100/\
                           agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < \
                            params['annual_ptnl_ret_lim']: # Covered return Requirement! 
                    continue
                if float(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'])*(-2) > \
                params['prob_touch_lim']: # Probability of touching parameter      + or - ???
                    continue
                support = 0
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']<supp1 and agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']>supp2:
                    support = 1
#Commented code checks for 24% annual return. If required in future
#                 value = (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']*365*100)/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']*agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'])
#                 if value<15:
#                     continue
    
                # Construct a dictionary object characteristics we want and append to list
                target = {'stock':symbol,
                         'contract':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
                         'strike_price':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
                         'stock_price':stock_price,
                         'days_to_exp':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],
                         'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
                         'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
                         'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
                         'prob_OTM': 1 + float(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']), # + or - ???
                         '% OTM': 1-agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']/stock_price,
                         'ptnl_ret': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] - \
                          max(0, (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - \
                            stock_price)))/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']- \
                                           agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']),
                         'annual_ptnl_ret': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] - \
                          max(0, (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - \
                            stock_price)))/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']- \
                                           agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']) * 365 * 100/\
                           agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],

                         'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*(-2),
                         'btwn_supports': support}# + or - ???
                
#                 print("The value of accepted delta",target['prob_OTM'])
                target_list.append(target)
    df = pd.DataFrame(target_list)
    if(df.empty):
        print('Got empty')
        return df
    df = df.set_index('contract', drop = True)
    return df



# Following function returns price data for a stock which can be used to construct a P&F chart
# def return_candle_data(symbol):
#     candle_url = 'https://api.tdameritrade.com/v1/marketdata/'+symbol+'/pricehistory'
#     candle_header = {'Authorization': get_auth_token()}
#     # Vary the properties below depending on how often you want the data
#     # Ideally, we want to set up a web socket to continuously stream this data?
#     period_type = 'day'
#     period = 2
#     freq_type = 'minute'
#     freq = 1
#     candle_payload = {'periodType': period_type, 'period': period, 'frequencyType': freq_type, 'frequency': freq}
#     raw_candle_data = requests.get(candle_url, params = candle_payload, headers = candle_header).json()
#     df = pd.DataFrame(raw_candle_data['candles'])
#     return df


def report(stock_list, Option_Data, put_parameters):
    # Select by WOO ratings and earnings dates criteria
    
    df_top_five_all =  pd.DataFrame()
    
    d0 = date.today()
    
    for row in tqdm(stock_list.iterrows()):
        item = row[1]['Symbol']
        supp1 = row[1]['Support1']
        supp2 = row[1]['Support2']
        name = row[1]['Company Name']
        print(item, end=" ")
        df = processing(item, Option_Data, supp1,supp2, put_parameters, "PUT")
        if(not df.empty):
            df_top_five = df.sort_values(by=['annual_ptnl_ret'], ascending=False)
            df_top_five = df_top_five.head(5)
            df_top_five['company name'] = name
            if df_top_five_all.empty:
                df_top_five_all = df_top_five
            else:
                df_top_five_all = df_top_five_all.append(df_top_five)
        #time.sleep(5)
                
    if(not df_top_five_all.empty):
        df_top_five_all = df_top_five_all.sort_values(by=['btwn_supports','stock','annual_ptnl_ret'], ascending=[False,True,False])
        
        df_top_five_all['Days to Earnings'] = '!' # In case earnings date data is not avaialbe, avoid reporting the contracts.
        
        df_top_five_all['Risk level'] = put_parameters['label']
        
        for index, row in df_top_five_all.iterrows():
            
            try:
                ed = next_earnings_date(row['stock']).date()
                delta = ed - d0 #compute dates from today to earnings date
                #print(delta.days)
                print('The next earnings date of {} is {}.'.format(row['stock'],ed))
           
                #if delta.days>0: # Yahoo may report earning dates that just passed.
                d_days = delta.days - row['days_to_exp']                

                if (d_days <= 0):
                    df_top_five_all = df_top_five_all.drop(index, axis=0)
                else:
                    df_top_five_all.at[index, 'Days to Earnings']= delta.days
            except:
                print(f"{row['stock']}: Error happens in finding the next earnings data.")
                df_top_five_all = df_top_five_all.drop(index, axis=0)
                pass

    else:
        print("No contracts available in the {} criteria.".format(put_parameters['label']))
    
    return df_top_five_all    
               
        
    
        

In [455]:
stock_list = get_final_list() # Need to populate this array with stock list we derive from technical indicators
#earnings_df = earnings_dates()
#print('Earnings List:',earnings_df)
#print('Stock List Before:', stock_list)

# stock_list.to_csv(r'C:\Users\User\Box\WOOatUB\YashaAbhinavRohit\Stock_List.csv')

# stock_list.to_csv(r'C:\Users\User\Box\WOOatUB\YashaAbhinavRohit\ContractsData\\'+ datetime.today().strftime('%Y-%m-%d')+'.csv')

if stock_list.empty == 0:
    stock_list.sort_values(by=['Score'], ascending = False)    
    stock_list = get_close_price(stock_list)
    print('Final stock list is done.')

Final stock list is done.


In [456]:
#Sanity
stock_list.to_csv("yfin_stock_list.csv")

In [457]:
# #df['EA'][0]
# Date_Keys = []
# Agg_Data = {}

# for (k,v) in df['ULTA'][0].items():
#     date_keys.append([k,list(v.keys())])
#     agg_data[k] = v
    
    
# Set selection parameters
safe_put_parameters = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, \
                       'max_vol': 1000000000, '%OTM_lim': 0.0, 'prob_OTM_lim': 0.95, 'annual_ptnl_ret_lim': 0.01, \
                       'prob_touch_lim': 0.9, 'label': 'safe'}
mod_put_parameters = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, \
                      'max_vol': 1000000000, '%OTM_lim': 0.0, 'prob_OTM_lim': 0.90, 'annual_ptnl_ret_lim': 0.01, \
                      'prob_touch_lim': 0.9, 'label': 'moderate'}
risky_put_parameters = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, \
                        'max_vol': 1000000000, '%OTM_lim': 0.0, 'prob_OTM_lim': 0.80, 'annual_ptnl_ret_lim': 0.01, \
                        'prob_touch_lim': 0.9, 'label': 'risky'}

# Collect option data

data = {}

for row in stock_list.iterrows():
    item = row[1]['Symbol'] 
    #print(item)
    data_entry = get_raw_option_data('PUT', item)
    # print(data_entry)
    data.update({item: data_entry})
    # time.sleep(2)
    
Option_Data = pd.DataFrame(data)

# Generating recommendations
df_risky = report(stock_list, Option_Data, risky_put_parameters)

df_mod = report(stock_list, Option_Data, mod_put_parameters)

df_safe = report(stock_list, Option_Data, safe_put_parameters)


Frames = [df_risky, df_mod, df_safe]

# Collect option data

# data = {}

# for row in df_control.iterrows():
#     item = row[1]['Symbol'] 
#     #print(item)
#     data_entry = get_raw_option_data('PUT', item)
#     data.update({item: data_entry})
#     # time.sleep(2)
    
# Option_Data = pd.DataFrame(data)


# # Generating control group

# df_risky = report(df_control, Option_Data, risky_put_parameters)

# df_mod = report(df_control, Option_Data, mod_put_parameters)

# df_safe = report(df_control, Option_Data, safe_put_parameters)


# Frames_Control = [df_risky, df_mod, df_safe]





0it [00:00, ?it/s]

AJG CBOE Got empty
COP 

17it [00:00, 283.33it/s]
17it [00:00, 448.05it/s]
17it [00:00, 548.35it/s]

MRO MPC OXY FTV Got empty
IEX Got empty
PWR SNA Got empty
GWW APD MCK GPC Got empty
MSI PTC Got empty
MNST The next earnings date of GWW is 2023-02-02.
The next earnings date of MCK is 2023-01-31.
The next earnings date of MNST is 2022-11-03.
The next earnings date of MNST is 2022-11-03.
The next earnings date of MSI is 2022-11-03.
The next earnings date of PWR is 2022-11-03.
The next earnings date of PWR is 2022-11-03.
The next earnings date of AJG is 2023-01-25.
The next earnings date of APD is 2023-02-02.
The next earnings date of APD is 2023-02-02.
The next earnings date of APD is 2023-02-02.
The next earnings date of APD is 2023-02-02.
The next earnings date of APD is 2023-02-02.
The next earnings date of COP is 2023-02-01.
The next earnings date of COP is 2023-02-01.
The next earnings date of COP is 2023-02-01.
The next earnings date of COP is 2023-02-01.
The next earnings date of COP is 2023-02-01.
The next earnings date of MCK is 2023-01-31.
The next earnings date of MCK is 202

In [458]:
def final_result(frames):
    '''Generate final results'''
    df = pd.concat(frames)
    df.reset_index(inplace = True)

    for i in df.index:
        '''Managing the duplcation of contracts at different risky levels'''
        #print(df.iloc[i]['Risk level']) #.values[0] == 'risky')
        if df.loc[i,'Risk level'] in  ['moderate', 'Moderate']:
            try:
                index_names = df[(df['Risk level'].isin(['Risky', 'risky'])) & \
                                 (df['contract'] == df.loc[i,'contract'])].index
                df.drop(index_names, inplace = True)

            except:
                print('Something is wrong when removing moderate contracts from risky ones.')

    for i in df.index:
        if df.loc[i,'Risk level'] in ['Safe', 'safe']:
            try:
                index_names = df[(df['Risk level'].isin(['moderate', 'Moderate']))\
                                 & (df['contract'] == df.loc[i,'contract'])].index
                df.drop(index_names, inplace = True)
            except:
                print('Something is wrong when removing safe contracts from moderate ones.')

    df['Date of recommendation'] = datetime.today().strftime('%Y-%m-%d') # add the date of recommendation column

    df.reset_index(inplace = True)

    #df.set_index('contract', inplace = True)

    if len(df) == 0:
        no_rec_dict = {'index': '0000',
                       'contract': ['No Recommendations'],
                       'stock': ['No Recommendations'],
                       'strike_price':[np.nan],
                       'stock_price':[np.nan],
                       'days_to_exp':[np.nan],
                       'bid': [np.nan],
                       'ask': [np.nan],
                       'last': [np.nan],
                       'prob_OTM': [np.nan],
                       '% OTM': [np.nan],
                       'ptnl_ret': [np.nan],
                       'annual_ptnl_ret': [np.nan],
                       'prob_touch': [np.nan],
                       'btwn_supports': [np.nan],
                       'company name': [''],
                       'Days to Earnings': [np.nan],
                       'Risk level': [''], 
                       'Date of recommendation': [datetime.today().strftime('%Y-%m-%d')]               
                         }
        df_no_rec = pd.DataFrame.from_dict(no_rec_dict)
        #print(df)
        df = df.append(df_no_rec)
    return df

result_df = final_result(Frames)
print(result_df)

# result_df.to_csv(r'C:\Users\User\Box\WOOatUB\YashaAbhinavRohit\ContractsData\'' + datetime.today().strftime('%Y-%m-%d')+'_final_list.csv')

# result_df_control = final_result(Frames_Control)
# result_df_control.to_csv(r'C:\Users\User\Box\WOOatUB\YashaAbhinavRohit\ContractsData\'' + datetime.today().strftime('%Y-%m-%d')+'_final_list_control.csv')

    index          contract stock  strike_price  stock_price  days_to_exp  \
0       0    GWW_111822P560   GWW         560.0      593.640           13   
1       1  MCK_111122P382.5   MCK         382.5      389.250            6   
2       2    AJG_111822P175   AJG         175.0      189.025           13   
3       3    APD_111822P260   APD         260.0      276.375           13   
4       4    APD_121622P250   APD         250.0      276.375           41   
5       6    APD_121622P240   APD         240.0      276.375           41   
6       8    COP_111122P126   COP         126.0      130.875            6   
7       9    COP_111122P125   COP         125.0      130.875            6   
8      10    COP_111122P124   COP         124.0      130.875            6   
9      11    COP_111122P123   COP         123.0      130.875            6   
10     12    COP_120222P120   COP         120.0      130.875           27   
11     13    MCK_111122P380   MCK         380.0      389.250            6   

In [459]:
#Sanity
result_df.to_csv("yfin_result.csv")

In [460]:

# import gspread
# from oauth2client.service_account import ServiceAccountCredentials


In [461]:
# Update to Google Sheet 

# Mert's Google Sheet
# scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
# creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
# client = gspread.authorize(creds)

# print("Started fetching data")
# sheet =  client.open("Algo_Recommendations").worksheet("All_The_Recommendations")
# print("Data fetching completed")

# for i in result_df.index:
#     values = [ str(i), str(result_df.loc[i,'contract']), str(result_df.loc[i,'% OTM']), \
#               str(result_df.loc[i,'annual_ptnl_ret']), str(result_df.loc[i,'ask']), str(result_df.loc[i,'bid']), \
#               str(result_df.loc[i,'btwn_supports']), str(result_df.loc[i,'days_to_exp']), str(result_df.loc[i,'last']), 
#               str(result_df.loc[i,'prob_OTM']), str(result_df.loc[i,'prob_touch']), str(result_df.loc[i,'ptnl_ret']), 
#               str(result_df.loc[i,'stock']), str(result_df.loc[i,'stock_price']), str(result_df.loc[i,'strike_price']), str(result_df.loc[i,'company name']), 
#               str(result_df.loc[i,'Days to Earnings']), str(result_df.loc[i,'Risk level']), str(result_df.loc[i,'Date of recommendation'])]
#     #print(values)
#     sheet.append_row(values) # we cannot send more than 100 contracts per 5 minutes



In [462]:
#  Mert's Google Server

#df=pd.read_csv('2020-08-11_final_list.csv')
# new_df=result_df.rename(columns={'% OTM':'percentage_OTM', 'company name':'company_name', 
#                           'Days to Earnings': 'days_to_earnings', 'Risk level': 'risk_level', 
#                           'Date of recommendation': 'date_of_recommendation'})
# print(new_df)


# def convertDateToTDAmeritradeDate(date):
#     month = date[0] + date[1];
#     day = date[2] + date[3];
#     year = "20" + date[4] + date[5];

#     return year + "-" + month + "-" + day + " 16:00:00";


# def getContractData(contract):
#     name_and_rest = contract.split("_");
#     date_and_strike = name_and_rest[1].split("P");
#     date = convertDateToTDAmeritradeDate(date_and_strike[0]);
#     #print(date)
#     return date;

# #datetime_str = '08/11/18 13:00:00'
# #datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')

# if new_df.iloc[0]['contract'] != 'No Recommendations':
#     new_df['expiration_date'] = new_df['contract'].apply(lambda x:getContractData(x))
#     #2020-08-14 16:00:00
#     new_df['expiration_date'] = new_df['expiration_date'].apply(lambda x:datetime.strptime(x+'-UTC', '%Y-%m-%d %H:%M:%S-%Z'))
#     #new_df['date_of_recommendation'] = new_df['date_of_recommendation'].apply(lambda x:datetime.strptime(x+' 05:00:00-UTC', '%m/%d/%Y %H:%M:%S-%Z'))
#     new_df['date_of_recommendation'] = new_df['date_of_recommendation'].apply(lambda x:datetime.strptime(x+' 05:00:00-UTC', '%Y-%m-%d %H:%M:%S-%Z'))
#     #new_df.dtypes
# else:
#     new_df['expiration_date'] = ''
#     new_df['date_of_recommendation'] = new_df['date_of_recommendation'].apply(lambda x:datetime.strptime(x+' 05:00:00-UTC', '%Y-%m-%d %H:%M:%S-%Z'))
#     #new_df.dtypes
    

# len_of_new_df = len(new_df)

# new_df = new_df.transpose()
# new_df_dict = new_df.to_dict()
    
#new_df_dict

In [463]:
# import firebase_admin
# from firebase_admin import credentials
# from firebase_admin import firestore

# # Use a service account
# cred = credentials.Certificate('key.json')
# firebase_admin.initialize_app(cred)
# db = firestore.client()

In [464]:
# all_the_rec = db.collection(u'test6').where(u"id", u'>', 1000).order_by(u"id", "DESCENDING").limit(1)
# newId = -100000

# docs = all_the_rec.stream()

# for doc in docs:
#     newId = int(doc.id) + 1
    
# print(newId)

# for i in range(0, len_of_new_df, 1):
#     contract_dict = new_df_dict[i]
#     doc_id = newId + i
#     contract_dict['id'] = doc_id
#     contract_dict['expired'] = False
#     #print(contract_dict)
#     db.collection(u'test6').document(str(doc_id)).set(contract_dict)



## Future Scope Functions

In [465]:
# def get_sector_ratings(etf,sector_sma_rating,sector_macd_rating,sector_macd_slope):
    
#     ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
#     data, meta_data = ts.get_daily_adjusted(symbol=etf, outputsize='full')
#     myData = data.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
#     #print(myData.head())
#     myDataSimple=myData[['Close']].copy()
#     myDataSimple['20d'] = np.round(myDataSimple.Close.rolling(window =20, center = False).mean(),2)
#     myDataSimple['50d'] = np.round(myDataSimple.Close.rolling(window =50, center = False).mean(),2)
#     myDataSimple['200d'] = np.round(myDataSimple.Close.rolling(window =200, center = False).mean(),2)
#     sma_rating_today=SMA_rating(myDataSimple)
#     macd_rating_today,slope=MACD_rating(myDataSimple)
#     time.sleep(12)
#     return sma_rating_today,macd_rating_today,slope   


# for etf in sector_etf_list:
#     #print(etf)
#     sector_sma_rating[etf],sector_macd_rating[etf],sector_macd_slope[etf]=get_sector_ratings(etf,sector_sma_rating,sector_macd_rating,sector_macd_slope)
    

In [466]:
# def get_intra_day_data():
#     ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas')
#     intra_day_data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
#     intra_day_data = intra_day_data.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
#     intra_day_data['20m'] = np.round(intra_day_data.Close.rolling(window =20, center = False).mean(),2)
#     intra_day_data['50m'] = np.round(intra_day_data.Close.rolling(window =50, center = False).mean(),2)
#     intra_day_data['200m'] = np.round(intra_day_data.Close.rolling(window =200, center = False).mean(),2)
#     return intra_day_data

In [467]:
#SMA rating
# def SMA_rating_intra_day(data_simple):
# #     start_date = date(1998,1,2)
# #     yesterday = (datetime.now() - timedelta(days=1))
# #     end_date = date(yesterday.year,yesterday.month,yesterday.day)
#     sma_rating = 0
#     sma_ratings = []
#     for index,val in enumerate(data_simple['Close']):
#         if data_simple['Close'][index] > data_simple['50m'][index] > data_simple['200m'][index]:
#             sma_rating=1
#         elif data_simple['50m'][index] > data_simple['Close'][index]> data_simple['200m'][index]:
#             sma_rating=2
#         elif data_simple['50m'][index] > data_simple['200m'][index] > data_simple['Close'][index]:
#             sma_rating=3
#         elif data_simple['200m'][index] > data_simple['50m'][index] > data_simple['Close'][index]:
#             sma_rating=4
#         elif data_simple['200m'][index]> data_simple['Close'][index] > data_simple['50m'][index]:
#             sma_rating=5
#         elif data_simple['Close'][index] > data_simple['200m'][index] > data_simple['50m'][index]:
#             sma_rating=6
#         elif data_simple['Close'][index] > data_simple['50m'][index] > data_simple['200m'][index]:
#             sma_rating=7
#         sma_ratings.append(sma_rating)
#     return sma_ratings

In [468]:
# def MACD_rating_intra_day(data_simple):
#         data_simple['26 ema']=data_simple.Close.ewm(span=26).mean()
#         data_simple['12 ema']=data_simple.Close.ewm(span=12).mean()
#         data_simple['9 ema']=data_simple.Close.ewm(span=9).mean()
#         data_simple['MACD'] = (data_simple['12 ema'] - data_simple['26 ema'])
#         data_simple['signal_line']=data_simple.MACD.ewm(span=9).mean()
#         data_simple['hist']=(data_simple['MACD']-data_simple['signal_line'])
#         #MACD rating
#         index = len(data_simple.index)-1
#         macd_rating = 0
#         macd_ratings = []
#         for index,val in enumerate(data_simple['Close']):
#             if data_simple['MACD'][index] > data_simple['signal_line'][index] and data_simple['signal_line'][index] > 0:
#                 macd_rating=1
#             elif data_simple['signal_line'][index] > data_simple['MACD'][index] and data_simple['MACD'][index] > 0:
#                 macd_rating=2
#             elif data_simple['MACD'][index] < 0 and 0 < data_simple['signal_line'][index]:
#                 macd_rating=3
#             elif data_simple['MACD'][index] < data_simple['signal_line'][index] and data_simple['signal_line'][index]< 0:
#                 macd_rating=4
#             elif data_simple['signal_line'][index] < data_simple['MACD'][index] and data_simple['MACD'][index] < 0:
#                 macd_rating=5
#             elif data_simple['MACD'][index] > 0 and 0 > data_simple['signal_line'][index]:
#                 macd_rating=6
#             macd_ratings.append(macd_rating)
#         return macd_ratings